<a href="https://colab.research.google.com/github/Temerius/GenAI/blob/main/tarot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
import requests
import re
import sys
import json

from random import choice
from bs4 import BeautifulSoup
from openai import AzureOpenAI
from google.colab import userdata

In [3]:
def get_links(response):

    soup = BeautifulSoup(response.content, 'html.parser')
    card_links = []

    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith('/tarot/cards/'):
            card_links.append("https://www.tarot.com"+href)

    # for link in card_links:
    #     print(link)

    return card_links

In [4]:
def get_card_info(url):

    response = requests.get(url)
    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')
        meanings_dict = {}
        headers_to_find = ['upright', 'reversed', 'advice', 'love', 'career', 'yesno']

        for header in headers_to_find:
            header_tag = soup.find('h3', id=header)
            if header_tag:
                paragraphs = []
                for sibling in header_tag.find_next_siblings():
                    if sibling.name != 'p':
                        break
                    paragraphs.append(sibling.get_text(strip=True))
                meanings_dict[header] = paragraphs[0]

        # for key, value in meanings_dict.items():
        #     print(f"{key}: {value}")

    else:
        print(f"Failed to retrieve data: {response.status_code}")

    return meanings_dict if meanings_dict else ""

In [5]:
url = "https://www.tarot.com/tarot/cards"

response = requests.get(url)
card_names = []
tarot_cards = {}

if response.status_code == 200:

    card_links = get_links(response)
    for card_link in card_links:
        card_names.append(card_link.split('/')[-1])

    for index, card_link in enumerate(card_links):
        card_info = get_card_info(card_link)
        tarot_cards[card_names[index]] = card_info

    # for card, value in tarot_cards.items():
    #     print(f"Card: {card}, value: {value}")

else:
    print(f"Failed to retrieve data: {response.status_code}")

In [6]:
sys.getsizeof(tarot_cards)

2272

In [8]:
def draw_cards(card_names, num=3):
    if num > len(card_names):
        raise ValueError("Number of cards to draw exceeds the available cards.")

    drawn_cards = []
    dup_check = set()

    for _ in range(num):
        card = choice(card_names)
        while card in dup_check:
            card = choice(card_names)
        dup_check.add(card)
        is_reversed = choice([True, False])
        drawn_cards.append((card, is_reversed))

    return drawn_cards

In [9]:
draw_cards(card_names)

[('eight-of-coins', False), ('ten-of-wands', True), ('the-magician', True)]

In [10]:
def get_selected_cards_info(selected_cards, question_type):
    if question_type == "general":
        question_type = "upright"
    selected_cards_info = []
    for card, reversed in selected_cards:
        if reversed:
           selected_cards_info.append((card, tarot_cards[card]["reversed"]))
        else:
            selected_cards_info.append((card, tarot_cards[card][question_type]))
    return selected_cards_info

In [22]:
api_key = userdata.get('AZURE_OPENAI_API_KEY')
endpoint = userdata.get('AZURE_OPENAI_ENDPOINT')

In [23]:
max_tokens_cls = 50
max_tokens_tarot = 550
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01"
)

user_info = "Name: John Doe, Date of Birth: January 15, 1990"

messages_cls = [
    {"role": "system", "content": "You are a helpful assistant that classifies user questions into one of the categories: 'advice', 'love', 'career', 'yesno', or 'general'."},
    {"role": "user", "content": "What should I do to improve my relationship?"},
    {"role": "assistant", "content": "advice"},
    {"role": "user", "content": "Will I meet someone special soon?"},
    {"role": "assistant", "content": "yesno"},
    {"role": "user", "content": "How can I get a promotion at work?"},
    {"role": "assistant", "content": "career"},
    {"role": "user", "content": "What can I do to find true love?"},
    {"role": "assistant", "content": "love"},
    {"role": "user", "content": "What should I consider before making a big decision?"},
    {"role": "assistant", "content": "general"}
]

messages_tarot = [
    {"role": "system", "content": "You are a tarot reader who provides insightful advice based on the user's question, the three drawn tarot cards, and the user's information. Your response should be thoughtful, relevant to the question type, and concise, ideally within 500 characters."}
]

while True:
    user_question = input("User (q/Q to quit): ")
    if user_question in {"q", "Q"}:
        print("Ai: Byebye")
        break

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_cls + [{"role": "user", "content": f"{user_question}"}],
        max_tokens=max_tokens_cls
    )

    question_type = response.choices[0].message.content
    print(f"Question type: {question_type}")

    if question_type in {"advice", "love", "career", "yesno", "general"}:
        selected_cards = draw_cards(card_names)
        print(selected_cards)

        selected_cards_info = get_selected_cards_info(selected_cards, question_type)

        user_prompt = (
            f"User question: {user_question}\n"
            f"User Information: {user_info}\n"
            f"Question type: {question_type}\n"
            f"Selected tarot cards and their meanings:\n"
            f"- {selected_cards_info[0][0]}: {selected_cards_info[0][1]}\n"
            f"- {selected_cards_info[1][0]}: {selected_cards_info[1][1]}\n"
            f"- {selected_cards_info[2][0]}: {selected_cards_info[2][1]}"
        )

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages_tarot + [{"role": "user", "content": f"{user_prompt}"}],
            max_tokens=max_tokens_tarot
        )
        print(response.choices[0].message.content)

User (q/Q to quit): Is now a good time to invest in stocks?
Question type: yesno
[('three-of-coins', False), ('six-of-swords', True), ('the-high-priestess', True)]
John, the tarot suggests that now is indeed a good time to invest in stocks (Three of Coins). Your collaborative efforts could lead to fruitful results. However, the Six of Swords reversed warns you to stay grounded and rational in your decisions; don't let emotions cloud your judgment. The reversed High Priestess indicates a need to balance your inner thoughts with external realities; engage with others for insights and support. Trust your skills and seek guidance from those around you, but remain objective in your investments.
User (q/Q to quit): q
Ai: Byebye
